In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
import sys
sys.path.append(os.path.abspath("../lib"))
from all import all_in_one

In [ ]:
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

spark = SparkSession \
    .builder \
    .appName("renovation") \
    .getOrCreate()

In [ ]:
# import files
# File location and type
file_location_array = [
    {"location": "../training/pred_dpe.csv", "name": "dpe", "delimiter": ","},
    {"location": "../training/dico.csv", "name": "dictionary", "delimiter": "\t"}
]
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
dataframes = {}

# The applied options are for CSV files. For other file types, these will be ignored.
for file in file_location_array:
    delimiter = file["delimiter"]
    file_location = file["location"]
    name = file["name"]
    dataframes[file["name"]] = (
        spark.read.format(file_type)
        .option("inferSchema", infer_schema)
        .option("header", first_row_is_header)
        .option("sep", delimiter)
        .load(file_location)
    )

In [ ]:
dpe = dataframes['dpe'].filter(F.col("heating_emission").cast("int").isNotNull()).withColumn('heating_emission', F.col('heating_emission').cast('int'))
dpe_training = dpe.drop('GES_emission').filter(F.col('DPE_consumption') < 700)
ges_training = dpe.drop('DPE_consumption').filter(F.col('GES_emission') < 200)
dictionary = dataframes['dictionary']

In [ ]:
# librairies
import numpy as np
import matplotlib.pyplot as plt

# usefull
from copy import deepcopy

# linear
from sklearn.linear_model import Ridge

# random forest
from sklearn.ensemble import RandomForestRegressor

# XGboost
from xgboost.sklearn import XGBRegressor

# quickXGboost
from sklearn.ensemble import HistGradientBoostingRegressor

#hyperparams
from scipy.stats import uniform, randint

In [ ]:
# linear
from sklearn.linear_model import LogisticRegression

# random forest
from sklearn.ensemble import RandomForestClassifier

# XGboost
from xgboost.sklearn import XGBClassifier

# HistGboost
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
dpe_training_cat = dpe_training.withColumn(
    'DPE_consumption',
    F.when(F.col('DPE_consumption') <= 50, 0)
    .when((F.col('DPE_consumption') > 50) & (F.col('DPE_consumption') <= 90), 1)
    .when((F.col('DPE_consumption') > 90) & (F.col('DPE_consumption') <= 150), 2)
    .when((F.col('DPE_consumption') > 150) & (F.col('DPE_consumption') <= 230), 3)
    .when((F.col('DPE_consumption') > 230) & (F.col('DPE_consumption') <= 330), 4)
    .when((F.col('DPE_consumption') > 330) & (F.col('DPE_consumption') <= 450), 5)
    .otherwise(6)
)

ges_training_cat = ges_training.withColumn(
    'GES_emission',
    F.when(F.col('GES_emission') <= 5, 0)
    .when((F.col('GES_emission') > 5) & (F.col('GES_emission') <= 10), 1)
    .when((F.col('GES_emission') > 10) & (F.col('GES_emission') <= 20), 2)
    .when((F.col('GES_emission') > 20) & (F.col('GES_emission') <= 35), 3)
    .when((F.col('GES_emission') > 35) & (F.col('GES_emission') <= 55), 4)
    .when((F.col('GES_emission') > 55) & (F.col('GES_emission') <= 80), 5)
    .otherwise(6)
)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from show import show_matrix

In [ ]:
dpe_param_reg = {'l2_regularization': 0.9632366534401319, 'learning_rate': 0.039390694124254874, 'max_iter': 120}
ges_param_reg = {'l2_regularization': 0.11976213181925122, 'learning_rate': 0.2971581842219685, 'max_iter': 124}
dpe_param_cat = {'class_weight': 'balanced', 'max_depth': 6, 'n_estimators': 103}
ges_param_cat = {'class_weight': 'balanced', 'max_depth': 7, 'n_estimators': 108}

col_X_hot = []
col_y_ges = ['GES_emission']
col_y_dpe = ['DPE_consumption']

col_X_not_hot = [col[0] for col in ges_training.dtypes if col[0] not in col_X_hot + col_y_ges]

X = dpe_training.select(col_X_not_hot).toPandas().to_numpy()

y_ges_cat = ges_training_cat.select(col_y_ges).toPandas().to_numpy().ravel()
y_dpe_cat = dpe_training_cat.select(col_y_dpe).toPandas().to_numpy().ravel()
if 0 not in np.unique(y_ges_cat):
    y_ges_cat = y_ges_cat - 1
if 0 not in np.unique(y_dpe_cat):
    y_dpe_cat = y_dpe_cat - 1

y_ges_reg = ges_training.select(col_y_ges).toPandas().to_numpy().ravel()
y_dpe_reg = dpe_training.select(col_y_dpe).toPandas().to_numpy().ravel()

if X.shape[0] > 100000:
    np.random.seed(42)
    indexes = np.random.choice(X.shape[0], 100000, replace=False)
    X = X[indexes, :]
    y_ges_cat = y_ges_cat[indexes]
    y_dpe_cat = y_dpe_cat[indexes]
    y_ges_reg = y_ges_reg[indexes]
    y_dpe_reg = y_dpe_reg[indexes]

X = StandardScaler().fit_transform(X)

train_index, test_index = list(ShuffleSplit(n_splits=5, random_state=0, test_size=0.25).split(X))[0]


In [ ]:
ges_regressor = HistGradientBoostingRegressor(**ges_param_reg).fit(X[train_index, :], y_ges_reg[train_index])
dpe_regressor = HistGradientBoostingRegressor(**dpe_param_reg).fit(X[train_index, :], y_dpe_reg[train_index])
ges_classifier = RandomForestClassifier(**ges_param_cat).fit(X[train_index, :], y_ges_cat[train_index])
dpe_classifier = RandomForestClassifier(**dpe_param_cat).fit(X[train_index, :], y_dpe_cat[train_index])

y_pred_ges_regressor = ges_regressor.predict(X[test_index, :])
y_pred_dpe_regressor = dpe_regressor.predict(X[test_index, :])
y_pred_ges_classifier = ges_classifier.predict(X[test_index, :])
y_pred_dpe_classifier = dpe_classifier.predict(X[test_index, :])

y_true_ges = y_ges_cat[test_index]
y_true_dpe = y_dpe_cat[test_index]

In [ ]:
y_pred_ges_regressor_to_cat = (
    spark.createDataFrame(y_pred_ges_regressor, ['GES_emission'])
    .withColumn(
        'GES_emission',
        F.when(F.col('GES_emission') <= 5, 0)
        .when((F.col('GES_emission') > 5) & (F.col('GES_emission') <= 10), 1)
        .when((F.col('GES_emission') > 10) & (F.col('GES_emission') <= 20), 2)
        .when((F.col('GES_emission') > 20) & (F.col('GES_emission') <= 35), 3)
        .when((F.col('GES_emission') > 35) & (F.col('GES_emission') <= 55), 4)
        .when((F.col('GES_emission') > 55) & (F.col('GES_emission') <= 80), 5)
        .otherwise(6)
    )
    .toPandas()
    .to_numpy()
    .ravel()
)



y_pred_dpe_regressor_to_cat = (
    spark.createDataFrame(y_pred_dpe_regressor, ['DPE_consumption'])
    .withColumn(
        'DPE_consumption',
        F.when(F.col('DPE_consumption') <= 50, 0)
        .when((F.col('DPE_consumption') > 50) & (F.col('DPE_consumption') <= 90), 1)
        .when((F.col('DPE_consumption') > 90) & (F.col('DPE_consumption') <= 150), 2)
        .when((F.col('DPE_consumption') > 150) & (F.col('DPE_consumption') <= 230), 3)
        .when((F.col('DPE_consumption') > 230) & (F.col('DPE_consumption') <= 330), 4)
        .when((F.col('DPE_consumption') > 330) & (F.col('DPE_consumption') <= 450), 5)
        .otherwise(6)
    )
    .toPandas()
    .to_numpy()
    .ravel()
)